# PART 2. 작업형2

##### **머신러닝 프로세스**

1.   문제 정의
2.   라이브러리/ 데이터 불러오기
3.   탐색적 데이터 분석(EDA)
4.   데이터 전처리
5.   검증 데이터 나누기
6.   모델 학습 및 평가
  - 분류 : 정확도, 정밀도, 재현율, F1 스코어, ROC-AUC
  - 회귀 : MSE, MAE, 결정 계수, RSME,MSLE,RMSLE,MAPE
7.   예측 및 제출

In [182]:
from google.colab import files
uploaded = files.upload()

In [212]:
import pandas as pd
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [213]:
train.isnull().sum()
# age - f/ workclass -o/ occupation-o / hours.per.week	-f/ native.country	-o

,0
id,0
age,12
workclass,1662
fnlwgt,0
education,0
education.num,0
marital.status,0
occupation,1668
relationship,0
race,0


In [214]:
train.info()
train.describe(include='O')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29304 entries, 0 to 29303
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              29304 non-null  int64  
 1   age             29292 non-null  float64
 2   workclass       27642 non-null  object 
 3   fnlwgt          29304 non-null  int64  
 4   education       29304 non-null  object 
 5   education.num   29304 non-null  int64  
 6   marital.status  29304 non-null  object 
 7   occupation      27636 non-null  object 
 8   relationship    29304 non-null  object 
 9   race            29304 non-null  object 
 10  sex             29304 non-null  object 
 11  capital.gain    29304 non-null  int64  
 12  capital.loss    29304 non-null  int64  
 13  hours.per.week  29291 non-null  float64
 14  native.country  28767 non-null  object 
 15  income          29304 non-null  object 
dtypes: float64(2), int64(5), object(9)
memory usage: 3.6+ MB


,workclass,education,marital.status,occupation,relationship,race,sex,native.country,income
count,27642,29304,29304,27636,29304,29304,29304,28767,29304
unique,8,16,7,14,6,5,2,41,2
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,United-States,<=50K
freq,20451,9449,13466,3683,11845,25022,19578,26240,22263


In [186]:
target = train.pop('income')

In [187]:
target.value_counts()

,count
income,
<=50K,22263
>50K,7041


### 결측치 처리

In [215]:
# 결측치제거
print("제거전 : ",train.shape)
df= train.dropna()
print("제거후 : ",df.shape)
df.isnull().sum()

제거전 :  (29304, 16)
제거후 :  (27096, 16)


,0
id,0
age,0
workclass,0
fnlwgt,0
education,0
education.num,0
marital.status,0
occupation,0
relationship,0
race,0


In [216]:
# 최빈값으로 결측치 처리
m = train['workclass'].mode()[0]
train['workclass'] = train['workclass'].fillna(m)
test['workclass'] = test['workclass'].fillna(m)

n = train['native.country'].mode()[0]
train['native.country']=train['native.country'].fillna(n)
test['native.country']=test['native.country'].fillna(n)

train.isnull().sum()

,0
id,0
age,12
workclass,0
fnlwgt,0
education,0
education.num,0
marital.status,0
occupation,1668
relationship,0
race,0


In [217]:
# 결측치를 'X'로 채우기
train['occupation'] = train['occupation'].fillna('X')
test['occupation'] = test['occupation'].fillna('X')

In [218]:
# 평균값으로 결측치 채우기
value = train['age'].mean()
train['age']=train['age'].fillna(value)
test['age']=test['age'].fillna(value)

In [219]:
# 중앙값으로 결측치 채우기
value = int(train['hours.per.week'].median())
train['hours.per.week'] = train['hours.per.week'].fillna(value)
test['hours.per.week'] = test['hours.per.week'].fillna(value)

In [220]:
train.isnull().sum()

,0
id,0
age,0
workclass,0
fnlwgt,0
education,0
education.num,0
marital.status,0
occupation,0
relationship,0
race,0


### 이상치 처리

In [221]:
train.describe()

,id,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
count,29304.000000,29304.000000,2.930400e+04,29304.000000,29304.000000,29304.000000,29304.000000
mean,16264.027880,38.553223,1.897488e+05,10.080842,1093.858722,86.744506,40.434036
std,9384.518323,13.626020,1.055250e+05,2.570824,7477.435640,401.518928,12.321306
min,0.000000,-38.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,8145.750000,28.000000,1.177890e+05,9.000000,0.000000,0.000000,40.000000
50%,16253.500000,37.000000,1.783765e+05,10.000000,0.000000,0.000000,40.000000
75%,24374.250000,48.000000,2.370682e+05,12.000000,0.000000,0.000000,45.000000
max,32560.000000,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [222]:
train = train[train['age']>0]
train.describe()

,id,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
count,29301.000000,29301.000000,2.930100e+04,29301.000000,29301.000000,29301.000000,29301.000000
mean,16264.061568,38.560446,1.897516e+05,10.080885,1093.970718,86.753387,40.432716
std,9384.427564,13.607894,1.055250e+05,2.570948,7477.810241,401.538524,12.320876
min,0.000000,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,8146.000000,28.000000,1.177890e+05,9.000000,0.000000,0.000000,40.000000
50%,16254.000000,37.000000,1.783700e+05,10.000000,0.000000,0.000000,40.000000
75%,24374.000000,48.000000,2.370650e+05,12.000000,0.000000,0.000000,45.000000
max,32560.000000,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


### 인코딩
    범주형 데이터를 숫자로 변환
- 레이블 인코딩

- 원-핫 인코딩

In [223]:
train_oh = pd.get_dummies(train)
test_oh = pd.get_dummies(test)
print(train.shape,test.shape,train_oh.shape,test_oh.shape)
# 학습 컬럼값과 테스트 컬럼값이 다르면 불가

(29301, 16) (3257, 15) (29301, 109) (3257, 103)


In [226]:
y_train = train.pop("income")

In [224]:
data = pd.concat([train,test],axis=0)
data_oh = pd.get_dummies(data)
train_oh = data_oh.iloc[:len(train)]
test_oh = data_oh.iloc[len(train):]
train_oh.shape, test_oh.shape


TypeError: 'int' object is not callable

In [227]:
cols = train.select_dtypes(include='O').columns

from sklearn.preprocessing import LabelEncoder

for col in cols:
  le = LabelEncoder()
  train[col] = le.fit_transform(train[col])
  test[col] = le.transform(test[col])

train.head()

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,3331,34.0,6,177331,15,10,2,9,0,2,1,4386,0,40.0,38
1,19749,58.0,3,290661,11,9,2,2,0,4,1,0,0,40.0,38
2,1157,48.0,3,125933,15,10,6,3,4,2,0,0,1669,38.0,38
3,693,58.0,3,100313,15,10,2,10,0,4,1,0,1902,40.0,38
4,12522,41.0,3,195661,15,10,2,13,0,4,1,0,0,54.0,38


### 스케일링
    수치형 데이터의 범위를 조정하는 작업

In [228]:
cols =['age','fnlwgt','education.num','capital.gain','capital.loss','hours.per.week']
cols

['age',
 'fnlwgt',
 'education.num',
 'capital.gain',
 'capital.loss',
 'hours.per.week']

In [229]:
from sklearn.preprocessing import MinMaxScaler
# 데이터를 0과 1사이로 변환
scaler = MinMaxScaler()
display(train[cols].head(2))
train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.transform(test[cols])
display(train[cols].head(2))

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,34.0,177331,10,4386,0,40.0
1,58.0,290661,9,0,0,40.0


,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,0.232877,0.112092,0.600000,0.04386,0.0,0.397959
1,0.561644,0.189060,0.533333,0.00000,0.0,0.397959


In [230]:
from sklearn.preprocessing import StandardScaler
# 데이터를 평균이 0이고 표준편차가 1인 분포로 변환
scaler = StandardScaler()
display(train[cols].head(2))
train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.transform(test[cols])
display(train[cols].head(2))

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,0.232877,0.112092,0.600000,0.04386,0.0,0.397959
1,0.561644,0.189060,0.533333,0.00000,0.0,0.397959


,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,-0.335138,-0.117705,-0.031462,0.440247,-0.216056,-0.035121
1,1.428574,0.956277,-0.420430,-0.146298,-0.216056,-0.035121


In [231]:
from sklearn.preprocessing import RobustScaler
# 각 값의 중앙값을 빼고, 1사분위수(1Q)와 3사분위수(3Q)의 차이(IQR)로 나누는 방법
scaler = RobustScaler()
display(train[cols].head(2))
train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.transform(test[cols])
display(train[cols].head(2))

,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,-0.335138,-0.117705,-0.031462,0.440247,-0.216056,-0.035121
1,1.428574,0.956277,-0.420430,-0.146298,-0.216056,-0.035121


,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week
0,-0.15,-0.008711,0.000000,0.586545,0.0,0.0
1,1.05,0.941438,-0.333333,0.000000,0.0,0.0


###### 타켓 변수가 범주형 변수일 경우 레이블 인코딩

In [232]:
target.value_counts()

,count
income,
<=50K,22263
>50K,7041


In [233]:
t_target = target.map({'<=50K':0,'>50K':1})

In [205]:
t_target.value_counts()

,count
income,
0,22263
1,7041


In [204]:
t_target = target.replace('<=50K',0).replace('>50K',1)
t_target.value_counts()

<ipython-input-204-0f923c767102>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  t_target = target.replace('<=50K',0).replace('>50K',1)


,count
income,
0,22263
1,7041


### 검증 데이터 나누기 (학습, 검증)

In [234]:
from sklearn.model_selection import train_test_split

X_train,X_val,y_train,y_val = train_test_split(train,y_train,test_size=0.2,random_state=0)
X_train.shape,X_val.shape,y_train.shape,y_val.shape


((23440, 15), (5861, 15), (23440,), (5861,))

### **머신러닝 학습 및 평가**

In [235]:
from sklearn.ensemble import RandomForestClassifier
# 랜덤 포레스트 : 여러개의 의사결정 나무를 기반으로 한 앙상블 학습
rf = RandomForestClassifier(random_state= 0)
rf.fit(X_train,y_train)
# predict : 레이블(클래스) 반환 , predict_proba : 각 레이블에 속할 확률값 반환
pred = rf.predict_proba(X_val)

print(rf.classes_)
pred[:10]

[0 1]


array([[1.  , 0.  ],
       [1.  , 0.  ],
       [0.91, 0.09],
       [0.61, 0.39],
       [0.99, 0.01],
       [0.99, 0.01],
       [0.98, 0.02],
       [0.94, 0.06],
       [0.12, 0.88],
       [0.88, 0.12]])

### 평가지표

| 구분(분류모델) | 평가지표 | 결과 및 설명 |
| ------- | ------- | ------- |
| predict_proba()    | roc_auc    | 2차원 형태 : [0.4, 0.6], [0.7, 0.3], [0.3, 0.7]<br>각 클래스 예측 확률 반환    |
| predict    | 정확도, F1_Score,정밀도,재현율    | 1차원 형태 : [1, 0, 1] <br> 예측된 클래스(레이블)반환    |

In [237]:
from sklearn.metrics import roc_auc_score
roc_auc = roc_auc_score(y_val,pred[:,1])
print('roc_auc:',roc_auc)

roc_auc: 0.9176072778031817


In [240]:
# Accuracy(정확도) 평가
pred = rf.predict(X_val)
pred[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [241]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_val,pred)
print('accuracy: ',accuracy)

accuracy:  0.8686231018597509


In [243]:
from sklearn.metrics import f1_score
f1 = f1_score(y_val,pred)
print('f1_score',f1)

f1_score 0.6907630522088354


In [246]:
# LightGBM : 그래디언트 부스팅(Gradient Boosting) 앙상블 학습 알고리즘

import lightgbm as lgb
lgbmc = lgb.LGBMClassifier(random_state=0,verbose=-1)
lgbmc.fit(X_train,y_train)
pred = lgbmc.predict_proba(X_val)

roc_auc = roc_auc_score(y_val,pred[:,1])
print('roc-auc:',roc_auc)

pred = lgbmc.predict(X_val)
accuracy = accuracy_score(y_val,pred)
print('accuracy_score:',accuracy)

f1 = f1_score(y_val,pred)
print('f1_score',f1)

roc-auc: 0.9279535666686397
accuracy_score: 0.8771540692714553
f1_score 0.7158642462509865


In [248]:
pred = lgbmc.predict_proba(test)
pred

array([[0.89924007, 0.10075993],
       [0.97622077, 0.02377923],
       [0.9853122 , 0.0146878 ],
       ...,
       [0.93528667, 0.06471333],
       [0.98996872, 0.01003128],
       [0.96723292, 0.03276708]])

In [249]:
submit = pd.DataFrame({'pred':pred[:,1]})
submit.to_csv("result.csv",index=False)